In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, date_format

# =======================================================================
# 1. CONFIGURAÇÃO E INICIALIZAÇÃO DO SPARK
# =======================================================================
conf = SparkConf()
conf.set(
    "spark.jars.packages",
    "org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.11.901"
)
conf.set(
    "spark.hadoop.fs.s3a.aws.credentials.provider",
    "com.amazonaws.auth.InstanceProfileCredentialsProvider"
)

spark = SparkSession.builder.appName("ExternalDataIntegration").config(conf=conf).getOrCreate()

# =======================================================================
# 2. DEFINIÇÃO DOS CAMINHOS DOS ARQUIVOS DE ENTRADA E SAÍDA
# =======================================================================

# Arquivos de Entrada no bucket trusted
INPUT_PATHS = {
    # Arquivos com granularidade por hora/minuto (DATA_ISO)
    "atendimentos": "s3a://bucket-trusted-upa-connect-sofh/BasesExternas/Atendimentos/AtendimentosTratada.csv",
    "clima_tempo": "s3a://bucket-trusted-upa-connect-sofh/BasesExternas/ClimaTempo/ClimaTempoTratada.csv",
    
    # Arquivos com granularidade diária
    "febre_amarela": "s3a://bucket-trusted-upa-connect-sofh/BasesExternas/FebreAmarela/FebreAmarelaTratada.csv",
    "sindrome_gripal": "s3a://bucket-trusted-upa-connect-sofh/BasesExternas/SindromeGripal/sindromeGripalTratada.csv"
}

# Caminho de Saída no bucket client
FINAL_OUTPUT_DIR = "s3a://bucket-client-upa-connect-sofh/basesExternas"
FINAL_FILENAME = "BaseIntegrada.csv"
TEMP_STAGING_DIR = f"{FINAL_OUTPUT_DIR}/_temp_staging_integrated"

# Coluna de Junção Comum (Padronizada para a Data apenas - YYYY-MM-DD)
# ALTERADO: Nome da coluna de junção agora é 'DataRegistro'
JOIN_COLUMN = "DataRegistro"

# =======================================================================
# 3. LEITURA DOS DADOS E PREPARAÇÃO PARA JOIN
# =======================================================================

dataframes = {}

print("Iniciando a leitura e padronização dos DataFrames do bucket trusted...")
for name, path in INPUT_PATHS.items():
    print(f"Lendo {name} de: {path}")
    
    # Leitura de CSV
    df = spark.read.option("delimiter", ";") \
                   .option("header", "true") \
                   .option("encoding", "UTF-8") \
                   .csv(path)
    
    # Padronização da Coluna de Data
    if name in ["atendimentos", "clima_tempo"]:
        # Extrai apenas a data (YYYY-MM-DD) da coluna DATA_ISO (ISO 8601)
        df = df.withColumn(
            JOIN_COLUMN, 
            date_format(col("DATA_ISO"), "yyyy-MM-dd")
        ).drop("DATA_ISO")
        
    elif name == "febre_amarela":
        # Converte a coluna DT_IS (DD/MM/YYYY) para a data padronizada (YYYY-MM-DD)
        df = df.withColumn(
            JOIN_COLUMN, 
            date_format(to_date(col("DT_IS"), "dd/MM/yyyy"), "yyyy-MM-dd")
        ).drop("DT_IS")
        
        # REMOÇÃO: Coluna FEBRE_AMARELA_ID será removida no bloco de remoção.
        
    elif name == "sindrome_gripal":
        # Renomeia DATA_NOTIFICACAO_TRATADA para a coluna de junção padrão.
        df = df.withColumnRenamed("DATA_NOTIFICACAO_TRATADA", JOIN_COLUMN)
        
        # REMOÇÃO: Coluna SINDROME_GRIPAL_ID será removida no bloco de remoção.
    
    # Renomeia colunas para evitar conflitos (exceto a coluna de junção)
    for c in df.columns:
        if c.upper() != JOIN_COLUMN.upper():
            df = df.withColumnRenamed(c, f"{name.upper()}_{c}")
            
    # Seleciona o DataFrame com a nova coluna de junção
    dataframes[name] = df

print("Leitura e preparação concluídas.")
print(f"Chave de junção padronizada: {JOIN_COLUMN}")

# =======================================================================
# 4. REALIZAÇÃO DA JUNÇÃO (FULL OUTER JOIN)
# =======================================================================
print("\nIniciando a junção dos DataFrames...")

# Começa com o DataFrame de Atendimentos
df_base = dataframes["atendimentos"]

# Realiza a junção com os demais DataFrames usando FULL OUTER JOIN para manter todos os registros de datas
for name, df_join in dataframes.items():
    if name != "atendimentos":
        # Usamos full outer join para manter a maior granularidade possível (a data),
        # garantindo que as datas não existentes em um lado sejam mantidas.
        df_base = df_base.join(df_join, on=JOIN_COLUMN, how="full_outer")
        print(f"Junção com {name} concluída.")

print("Junção de todos os DataFrames concluída.")

# =======================================================================
# 5. REMOÇÃO DE COLUNAS DE ID E SELEÇÃO FINAL
# =======================================================================

# Colunas a serem removidas do resultado final
COLUNAS_PARA_REMOVER = [
    "FEBRE_AMARELA_ID",
    "SINDROME_GRIPAL_ID"
]

# Cria uma lista de colunas para manter, excluindo as colunas de ID
colunas_finais = [c for c in df_base.columns if c not in COLUNAS_PARA_REMOVER]

# Aplica a seleção
df_base = df_base.select(colunas_finais)

df_base.printSchema()
print(f"Total de linhas na base integrada: {df_base.count()}")

# =======================================================================
# 6. SALVANDO E RENOMEANDO O RESULTADO NO S3 (BUCKET CLIENT)
# =======================================================================

# 1. Escreve o resultado no caminho temporário
print(f"\nEscrevendo dados temporariamente em: {TEMP_STAGING_DIR}")

# NOTA: Coalesce(1) para garantir a geração de um único arquivo CSV.
df_base.coalesce(1).write \
    .option('delimiter', ';') \
    .option('header', 'true') \
    .option('encoding', 'UTF-8') \
    .mode('overwrite') \
    .csv(TEMP_STAGING_DIR)

# 2. Renomeia o arquivo gerado
try:
    # Acessa a classe 'Path' da JVM através do gateway do Spark
    Path = spark._jvm.org.apache.hadoop.fs.Path
    
    # Acessa a configuração do Hadoop
    hadoop_conf = spark._jsc.hadoopConfiguration()
    
    # Obtém o objeto FileSystem para o caminho temporário
    fs = Path(TEMP_STAGING_DIR).getFileSystem(hadoop_conf)

    # Encontra o arquivo gerado (part-00000-*.csv) dentro do diretório temporário
    list_status = fs.globStatus(Path(TEMP_STAGING_DIR + "/part-00000-*.csv"))

    if list_status:
        # Pega o caminho completo do arquivo gerado
        generated_file_path = list_status[0].getPath()

        # Define o caminho final e o nome específico para o arquivo
        final_output_path = Path(f"{FINAL_OUTPUT_DIR}/{FINAL_FILENAME}")

        # Renomeia (move) o arquivo para o caminho e nome definitivos
        fs.rename(generated_file_path, final_output_path)
        
        # 3. Deleta o diretório temporário (que ficou vazio) e outros arquivos de metadados
        fs.delete(Path(TEMP_STAGING_DIR), True) 
        
        print(f"\n✅ Base integrada salva e renomeada com sucesso para: {final_output_path}")

    else:
        print("\nErro: Não foi possível encontrar o arquivo CSV gerado (part-00000-*.csv) no caminho temporário.")

except Exception as e:
    print(f"\nOcorreu um erro durante a renomeação do arquivo no S3: {e}")

# Encerra a sessão Spark
spark.stop()


:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-50c6a22a-112e-4b08-ace9-f1efd16d5bc6;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 382ms :: artifacts dl 20ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	:: evicted modules:
	com.amazonaws#aws-java-sdk-bundle;1.11.901 by [com.amazonaws#aws-java-sdk-bundle;1.12.262] in [default]
	---------------------------------------------------------------------
	|     

Iniciando a leitura e padronização dos DataFrames do bucket trusted...
Lendo atendimentos de: s3a://bucket-trusted-upa-connect-sofh/BasesExternas/Atendimentos/AtendimentosTratada.csv


25/10/17 14:37:37 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


Lendo clima_tempo de: s3a://bucket-trusted-upa-connect-sofh/BasesExternas/ClimaTempo/ClimaTempoTratada.csv
Lendo febre_amarela de: s3a://bucket-trusted-upa-connect-sofh/BasesExternas/FebreAmarela/FebreAmarelaTratada.csv
Lendo sindrome_gripal de: s3a://bucket-trusted-upa-connect-sofh/BasesExternas/SindromeGripal/sindromeGripalTratada.csv
Leitura e preparação concluídas.
Chave de junção padronizada: DataRegistro

Iniciando a junção dos DataFrames...
Junção com clima_tempo concluída.
Junção com febre_amarela concluída.
Junção com sindrome_gripal concluída.
Junção de todos os DataFrames concluída.
root
 |-- DataRegistro: string (nullable = true)
 |-- ATENDIMENTOS_SEXO: string (nullable = true)
 |-- ATENDIMENTOS_TIPO_DE_UNIDADE: string (nullable = true)
 |-- ATENDIMENTOS_DESCRICAO_DO_PROCEDIMENTO: string (nullable = true)
 |-- ATENDIMENTOS_DESCRICAO_DO_CBO: string (nullable = true)
 |-- ATENDIMENTOS_NACIONALIDADE: string (nullable = true)
 |-- ATENDIMENTOS_TIPO_PROCEDIMENTO: string (nullabl

Total de linhas na base integrada: 1342365

Escrevendo dados temporariamente em: s3a://bucket-client-upa-connect-sofh/basesExternas/_temp_staging_integrated


25/10/17 14:38:07 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/10/17 14:38:16 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
25/10/17 14:38:17 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.



✅ Base integrada salva e renomeada com sucesso para: s3a://bucket-client-upa-connect-sofh/basesExternas/BaseIntegrada.csv


In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, date_format
import re  # <<< ADICIONADO: Importação para Regex

# =======================================================================
# 1. CONFIGURAÇÃO E INICIALIZAÇÃO DO SPARK
# =======================================================================
conf = SparkConf()
conf.set(
    "spark.jars.packages",
    "org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.11.901"
)
conf.set(
    "spark.hadoop.fs.s3a.aws.credentials.provider",
    "com.amazonaws.auth.InstanceProfileCredentialsProvider"
)

spark = SparkSession.builder.appName("ExternalDataIntegration").config(conf=conf).getOrCreate()

# --- INÍCIO: FUNÇÃO AUXILIAR SNAKE_CASE ---
def to_snake_case(name):
    """
    Converte uma string (CamelCase, UPPER_CASE, etc.) para snake_case.
    Ex: "DataRegistro" -> "data_registro"
    Ex: "ATENDIMENTOS_SEXO" -> "atendimentos_sexo"
    Ex: "CLIMA_TEMPO_PM2_5" -> "clima_tempo_pm2_5"
    """
    # Insere _ antes de letras maiúsculas (ex: DataRegistro -> Data_Registro)
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    # Insere _ antes de grupos de letras maiúsculas/números (ex: CBO, PM2_5)
    s2 = re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1)
    # Converte tudo para minúsculo
    return s2.lower()
# --- FIM: FUNÇÃO AUXILIAR SNAKE_CASE ---


# =======================================================================
# 2. DEFINIÇÃO DOS CAMINHOS DOS ARQUIVOS DE ENTRADA E SAÍDA
# =======================================================================

# Arquivos de Entrada no bucket trusted
INPUT_PATHS = {
    # Arquivos com granularidade por hora/minuto (DATA_ISO)
    "atendimentos": "s3a://bucket-trusted-upa-connect-sofh/BasesExternas/Atendimentos/AtendimentosTratada.csv",
    "clima_tempo": "s3a://bucket-trusted-upa-connect-sofh/BasesExternas/ClimaTempo/ClimaTempoTratada.csv",
    
    # Arquivos com granularidade diária
    "febre_amarela": "s3a://bucket-trusted-upa-connect-sofh/BasesExternas/FebreAmarela/FebreAmarelaTratada.csv",
    "sindrome_gripal": "s3a://bucket-trusted-upa-connect-sofh/BasesExternas/SindromeGripal/sindromeGripalTratada.csv"
}

# Caminho de Saída no bucket client
FINAL_OUTPUT_DIR = "s3a://bucket-client-upa-connect-sofh/basesExternas"
FINAL_FILENAME = "BaseIntegrada.csv"
TEMP_STAGING_DIR = f"{FINAL_OUTPUT_DIR}/_temp_staging_integrated"

# Coluna de Junção Comum (Padronizada para a Data apenas - YYYY-MM-DD)
# ALTERADO: Nome da coluna de junção agora é 'DataRegistro'
JOIN_COLUMN = "DataRegistro"

# =======================================================================
# 3. LEITURA DOS DADOS E PREPARAÇÃO PARA JOIN
# =======================================================================

dataframes = {}

print("Iniciando a leitura e padronização dos DataFrames do bucket trusted...")
for name, path in INPUT_PATHS.items():
    print(f"Lendo {name} de: {path}")
    
    # Leitura de CSV
    df = spark.read.option("delimiter", ";") \
                   .option("header", "true") \
                   .option("encoding", "UTF-8") \
                   .csv(path)
    
    # Padronização da Coluna de Data
    if name in ["atendimentos", "clima_tempo"]:
        # Extrai apenas a data (YYYY-MM-DD) da coluna DATA_ISO (ISO 8601)
        df = df.withColumn(
            JOIN_COLUMN, 
            date_format(col("DATA_ISO"), "yyyy-MM-dd")
        ).drop("DATA_ISO")
        
    elif name == "febre_amarela":
        # Converte a coluna DT_IS (DD/MM/YYYY) para a data padronizada (YYYY-MM-DD)
        df = df.withColumn(
            JOIN_COLUMN, 
            date_format(to_date(col("DT_IS"), "dd/MM/yyyy"), "yyyy-MM-dd")
        ).drop("DT_IS")
        
        # REMOÇÃO: Coluna FEBRE_AMARELA_ID será removida no bloco de remoção.
        
    elif name == "sindrome_gripal":
        # Renomeia DATA_NOTIFICACAO_TRATADA para a coluna de junção padrão.
        df = df.withColumnRenamed("DATA_NOTIFICACAO_TRATADA", JOIN_COLUMN)
        
        # REMOÇÃO: Coluna SINDROME_GRIPAL_ID será removida no bloco de remoção.
    
    # Renomeia colunas para evitar conflitos (exceto a coluna de junção)
    for c in df.columns:
        if c.upper() != JOIN_COLUMN.upper():
            df = df.withColumnRenamed(c, f"{name.upper()}_{c}")
            
    # Seleciona o DataFrame com a nova coluna de junção
    dataframes[name] = df

print("Leitura e preparação concluídas.")
print(f"Chave de junção padronizada: {JOIN_COLUMN}")

# =======================================================================
# 4. REALIZAÇÃO DA JUNÇÃO (FULL OUTER JOIN)
# =======================================================================
print("\nIniciando a junção dos DataFrames...")

# Começa com o DataFrame de Atendimentos
df_base = dataframes["atendimentos"]

# Realiza a junção com os demais DataFrames usando FULL OUTER JOIN para manter todos os registros de datas
for name, df_join in dataframes.items():
    if name != "atendimentos":
        # Usamos full outer join para manter a maior granularidade possível (a data),
        # garantindo que as datas não existentes em um lado sejam mantidas.
        df_base = df_base.join(df_join, on=JOIN_COLUMN, how="full_outer")
        print(f"Junção com {name} concluída.")

print("Junção de todos os DataFrames concluída.")

# =======================================================================
# 5. REMOÇÃO DE COLUNAS DE ID, SELEÇÃO FINAL E RENOMEAÇÃO SNAKE_CASE
# =======================================================================

# Colunas a serem removidas do resultado final
# ATENÇÃO: A lógica de prefixação (ex: "SINDROME_GRIPAL_") pode exigir
# que esta lista seja atualizada para (ex: "SINDROME_GRIPAL_SINDROME_GRIPAL_ID").
# O schema que você forneceu indica que as colunas na lista abaixo
# podem não ser os nomes corretos após a prefixação.
COLUNAS_PARA_REMOVER = [
    "FEBRE_AMARELA_ID",
    "SINDROME_GRIPAL_ID" 
    # Com base no seu schema, talvez devesse ser "SINDROME_GRIPAL_id"
    # ou "SINDROME_GRIPAL_SINDROME_GRIPAL_ID"
]

# Cria uma lista de colunas para manter, excluindo as colunas de ID
colunas_finais = [c for c in df_base.columns if c not in COLUNAS_PARA_REMOVER]

# Aplica a seleção
df_selecionado = df_base.select(colunas_finais)

# --- INÍCIO: MODIFICAÇÃO PARA SNAKE_CASE ---
print("\nConvertendo colunas finais para snake_case...")

# Gera a lista de expressões 'select' com alias
# Ex: [col("DataRegistro").alias("data_registro"), col("ATENDIMENTOS_SEXO").alias("atendimentos_sexo"), ...]
select_exprs = [
    col(c).alias(to_snake_case(c)) for c in df_selecionado.columns
]

# Aplica a renomeação final
df_base = df_selecionado.select(select_exprs)
# --- FIM: MODIFICAÇÃO PARA SNAKE_CASE ---


df_base.printSchema()
print(f"Total de linhas na base integrada: {df_base.count()}")

# =======================================================================
# 6. SALVANDO E RENOMEANDO O RESULTADO NO S3 (BUCKET CLIENT)
# =======================================================================

# 1. Escreve o resultado no caminho temporário
print(f"\nEscrevendo dados temporariamente em: {TEMP_STAGING_DIR}")

# NOTA: Coalesce(1) para garantir a geração de um único arquivo CSV.
df_base.coalesce(1).write \
    .option('delimiter', ';') \
    .option('header', 'true') \
    .option('encoding', 'UTF-8') \
    .mode('overwrite') \
    .csv(TEMP_STAGING_DIR)

# 2. Renomeia o arquivo gerado
try:
    # Acessa a classe 'Path' da JVM através do gateway do Spark
    Path = spark._jvm.org.apache.hadoop.fs.Path
    
    # Acessa a configuração do Hadoop
    hadoop_conf = spark._jsc.hadoopConfiguration()
    
    # Obtém o objeto FileSystem para o caminho temporário
    fs = Path(TEMP_STAGING_DIR).getFileSystem(hadoop_conf)

    # Encontra o arquivo gerado (part-00000-*.csv) dentro do diretório temporário
    list_status = fs.globStatus(Path(TEMP_STAGING_DIR + "/part-00000-*.csv"))

    if list_status:
        # Pega o caminho completo do arquivo gerado
        generated_file_path = list_status[0].getPath()

        # Define o caminho final e o nome específico para o arquivo
        final_output_path = Path(f"{FINAL_OUTPUT_DIR}/{FINAL_FILENAME}")

        # Renomeia (move) o arquivo para o caminho e nome definitivos
        fs.rename(generated_file_path, final_output_path)
        
        # 3. Deleta o diretório temporário (que ficou vazio) e outros arquivos de metadados
        fs.delete(Path(TEMP_STAGING_DIR), True) 
        
        print(f"\n✅ Base integrada salva e renomeada com sucesso para: {final_output_path}")

    else:
        print("\nErro: Não foi possível encontrar o arquivo CSV gerado (part-00000-*.csv) no caminho temporário.")

except Exception as e:
    print(f"\nOcorreu um erro durante a renomeação do arquivo no S3: {e}")

# Encerra a sessão Spark
spark.stop()

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8b7071a9-d202-476f-9f0b-fdab881ae3ca;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 555ms :: artifacts dl 20ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	:: evicted modules:
	com.amazonaws#aws-java-sdk-bundle;1.11.901 by [com.amazonaws#aws-java-sdk-bundle;1.12.262] in [default]
	---------------------------------------------------------------------
	|     

Iniciando a leitura e padronização dos DataFrames do bucket trusted...
Lendo atendimentos de: s3a://bucket-trusted-upa-connect-sofh/BasesExternas/Atendimentos/AtendimentosTratada.csv


25/10/25 22:49:51 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


Lendo clima_tempo de: s3a://bucket-trusted-upa-connect-sofh/BasesExternas/ClimaTempo/ClimaTempoTratada.csv
Lendo febre_amarela de: s3a://bucket-trusted-upa-connect-sofh/BasesExternas/FebreAmarela/FebreAmarelaTratada.csv
Lendo sindrome_gripal de: s3a://bucket-trusted-upa-connect-sofh/BasesExternas/SindromeGripal/sindromeGripalTratada.csv
Leitura e preparação concluídas.
Chave de junção padronizada: DataRegistro

Iniciando a junção dos DataFrames...
Junção com clima_tempo concluída.
Junção com febre_amarela concluída.
Junção com sindrome_gripal concluída.
Junção de todos os DataFrames concluída.

Convertendo colunas finais para snake_case...
root
 |-- data_registro: string (nullable = true)
 |-- atendimentos_sexo: string (nullable = true)
 |-- atendimentos_tipo_de_unidade: string (nullable = true)
 |-- atendimentos_descricao_do_procedimento: string (nullable = true)
 |-- atendimentos_descricao_do_cbo: string (nullable = true)
 |-- atendimentos_nacionalidade: string (nullable = true)
 |--

Total de linhas na base integrada: 1342365

Escrevendo dados temporariamente em: s3a://bucket-client-upa-connect-sofh/basesExternas/_temp_staging_integrated


25/10/25 22:50:22 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/10/25 22:53:23 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
25/10/25 22:53:24 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.



✅ Base integrada salva e renomeada com sucesso para: s3a://bucket-client-upa-connect-sofh/basesExternas/BaseIntegrada.csv
